群晖moments上传照片，无法自动遍历整个文件夹和子文件夹，所以只能一个文件夹一个文件夹进行，非常麻烦。
所以需要将一个大文件夹中的所有目标文件提取到一个位置，不存在子文件夹，这样可以方便上传。
但是这样面临的问题就是，比如对于iPhone照片，文件名规则就是IMG_xxxx.JPG/HEIC，会有文件名重合情况出现，要防止overwrite。

首先判断文件名中是否有日期信息，如果有，那就肯定不会出现overwrite情况。

上面方法太傻，换个方法，建立一个新的文件夹，以及此文件夹中的文件名list，每move一个文件进来，先判断是否存在同名文件，如果没有就move，将list增加该文件名。如果有同名情况，那么建立一个新的文件夹2，将该文件放入新文件夹。
这是个递归的过程。可以一次将所有文件处理完毕。

用用yield方式？

可以不是递归方式，一个foreach即可，所有文件处理完毕就算完成。

首先建立测试环境。
目标文件类型，参考群晖moments的支持文件格式，图像格式：BMP,GIF,JPEG,PNG,RAW（3fr,arw,cr2,cr3,crw,dcr,dng,erf,k25,kdc,mef,mos,mrw,nef,orf,ptx,pef,raf,raw,rw2,sr2,srf）,TIFF,HEIC
视频格式：3G2,3GP,ASF,AVI,DivX,FLV,M4V,MOV,MP4,MPEG,MPG,MTS,M2TS,M2T,QT,WMV,XviD
文件夹，子文件夹树状结构。
文件名，重名。

可以不用建立测试环境，只要没有删除，和overwrite即可。

首先，目标文件夹，提取所有影像文件list。

In [1]:
import glob, os
from pathlib import Path, WindowsPath, PurePosixPath
import pickle
from datetime import datetime
from PIL import Image
import exifread

src_dir = "../photos_mom/**"

def write_list(list_name,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(list_name, fp)
        print('Done writing list into a binary file')
        
# Read list to memory
def read_list(filename):
    # for reading also binary mode is important
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list


file_ext = ['jpg','jpeg','BMP','GIF','JPEG','PNG','RAW','3fr','arw',\
            'cr2','cr3','crw','dcr','dng','erf','k25','kdc','mef','mos',\
                'mrw','nef','orf','ptx','pef','raf','raw','rw2','sr2',\
                    'srf','TIFF','tif','HEIC','3G2','3GP','ASF','AVI','DivX',\
                        'FLV','M4V','MOV','MP4','MPEG','MPG','MTS','M2TS',\
                            'M2T','QT','WMV','XviD']
src_file = []

for filename in glob.iglob(pathname= src_dir, recursive=True):
    if os.path.isfile(filename) \
        and (Path(filename).suffix.lower()[1:] in file_ext or \
             Path(filename).suffix.upper()[1:] in file_ext or \
                Path(filename).suffix[1:] in file_ext):
        src_file.append(filename.replace('\\', '/'))
    else:
        print(filename)

../photos_mom\
../photos_mom\0
../photos_mom\1


得到了所有目标文件的列表。
下面也进行了文件的移动，但是结果的子文件夹还是有点多。

In [5]:
# toDir = "../photos_mom/"


# index = 0

# while src_file:
#     tar_dir = toDir+str(index)
#     os.makedirs(tar_dir)
#     index = index + 1
#     toDir_lst = []
#     for idx, filename in enumerate(src_file):
#         if os.path.basename(filename) not in toDir_lst:
#             # move to dir
#             # add to TO list
#             # remove from src list
#             os.rename(filename, tar_dir + '/' + os.path.basename(filename))
#             toDir_lst.append(os.path.basename(filename))
#             src_file.pop(idx)

一共4079个文件，为什么处理后，从最后向前文件夹中的文件数量，1，2，4，8，16，32，64，128，255，511，1020，2038个文件？
for循环中的src_file不可以在循环中pop，否则，数量不断的减半。

更改为下面的代码，仍然如此。src_file = src_file_t，实际上两个list同时指向一个对象list。应该这样src_file = src_file_t[:]

成功，达成目标。

他妈的，moments一次最多上传2000个文件。

In [2]:
toDir = "../photos_mom2/"

src_file_t = src_file[:]
index = 0

while src_file:
    tar_dir = toDir+str(index)
    os.makedirs(tar_dir)
    index = index + 1
    toDir_lst = []
    for idx, filename in enumerate(src_file):
        if idx >= 1999:
            break
        if os.path.basename(filename) not in toDir_lst:
            # move to dir
            # add to TO list
            # remove from src list
            os.rename(filename, tar_dir + '/' + os.path.basename(filename))
            toDir_lst.append(os.path.basename(filename))
            src_file_t.remove(filename)
    
    src_file = src_file_t[:]